In [5]:
import torchvision
import torch.nn as nn
import torch.nn.functional as nnf
import torch
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms.functional import to_tensor, normalize
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output
from torchvision import transforms, models
from torchvision.datasets import Food101
from torchvision import datasets, transforms
import statistics as st

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
print(torch.cuda.get_device_name(0))

def test_acc(net: nn.Module, test_loader: DataLoader):

  net.to(device)
  net.eval()
  
  total = 0
  correct = 0

  for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)
    total += labels.size(0)

    outputs = net(images)
    _, predicted = torch.max(outputs, 1)
    correct += (predicted == labels).sum().item()

  return correct / total * 100

def train_fn(epochs: int, train_loader: DataLoader,
             net: nn.Module, loss_fn: nn.Module, optimizer: optim.Optimizer, train_dataset_length):

  # losses = []
  # accuracies = []

  net.to(device)

  for e in range(epochs):
    net.train()
    running_loss = 0.0

    for images, labels in train_loader:
      images, labels = images.to(device), labels.type(torch.LongTensor).to(device)

      optimizer.zero_grad()
      outputs = net(images)
      loss = loss_fn(outputs, labels)
      loss.backward()
      optimizer.step()

      running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / train_dataset_length

    print(f"Loss: {epoch_loss}")

    # acc = test_acc(net, test_loader)
    # print(f"Acuratetea la finalul epocii {e + 1} este {acc:.2f}%")

    # losses.append(epoch_loss)
    # accuracies.append(acc)

    # torch.save(net.state_dict(), f'resnet50_multiplefclayers_adamw_epoch{e + 1}.pkl')

  # loss_graph(losses)
  # accuracy_graph(accuracies)

def loss_graph(losses):
    plt.plot(losses)

    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.show()

def accuracy_graph(accuracies):
    plt.plot(accuracies)

    plt.xlabel('epochs')
    plt.ylabel('accuracy')
    plt.show()

class CustomClassifier(nn.Module):
    def __init__(self):
        super(CustomClassifier, self).__init__()

        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.25),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.1),
            nn.Linear(128, 101)
        )

    def forward(self, x):
        return self.classifier(x)

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])
])

cuda
NVIDIA GeForce RTX 4060 Laptop GPU


In [6]:
# Adam

configs = [0.01, 0.005, 0.001, 0.0005, 0.0001]
mean_accuracies = []
i = 1

for lr in configs:

    print(f'Config {i}:')
    i += 1
    accuracies = []
    start = time.time()

    for split_number in range(1, 6):

        train_dir = f"C:/Users/Vlad Talpiga.VLR_PROJAMZ/OneDrive - Valrom Industrie SRL/Desktop/Facultate/IAVA/Proiect/Food Classifier Final/datasets/dataset_cross_validation/split{split_number}/train"
        val_dir = f"C:/Users/Vlad Talpiga.VLR_PROJAMZ/OneDrive - Valrom Industrie SRL/Desktop/Facultate/IAVA/Proiect/Food Classifier Final/datasets/dataset_cross_validation/split{split_number}/validation"
    
        train_dataset = datasets.ImageFolder(train_dir, transform=transform)
        val_dataset = datasets.ImageFolder(val_dir, transform=transform)

        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

        resnet = models.resnet50(pretrained=True)
        resnet = nn.Sequential(*list(resnet.children())[:-1])

        # Freeze all layers except the final classification layer
        for param in resnet.parameters():
            param.requires_grad = False

        resnet = nn.Sequential(
            resnet,
            CustomClassifier()
        )

        loss_fn = nn.CrossEntropyLoss()
        optimizer = optim.Adam(resnet.parameters(), lr=lr)

        print(f'Training split {split_number}')

        train_fn(1, train_loader, resnet, loss_fn, optimizer, len(train_dataset))

        acc = test_acc(resnet, val_loader)

        print(f'Accuracy: {acc:.2f}%')
        accuracies.append(acc)

        
    print(f'Duration of training: {(time.time() - start) / 60}')
    mean_accuracies.append(st.mean(accuracies))
    
    print(accuracies)
    print(st.mean(accuracies))

print(mean_accuracies)

Config 1:
Training split 1
Loss: 2.4320286382542977
Accuracy: 48.63%
Training split 2
Loss: 2.4306515398119934
Accuracy: 48.46%
Training split 3
Loss: 2.422784915442514
Accuracy: 46.91%
Training split 4
Loss: 2.4440801019951848
Accuracy: 47.32%
Training split 5
Loss: 2.4205038123555704
Accuracy: 46.99%
Duration of training: 50.885087764263154
[48.633663366336634, 48.45544554455445, 46.9059405940594, 47.31683168316832, 46.99009900990099]
47.66039603960396
Config 2:
Training split 1
Loss: 2.39433693319264
Accuracy: 49.95%
Training split 2
Loss: 2.3799793240575506
Accuracy: 49.31%
Training split 3
Loss: 2.3885394557159727
Accuracy: 47.96%
Training split 4
Loss: 2.4034485357114583
Accuracy: 50.16%
Training split 5
Loss: 2.39421308781841
Accuracy: 47.83%
Duration of training: 42.542802421251935
[49.945544554455445, 49.306930693069305, 47.96039603960396, 50.15841584158416, 47.83168316831683]
49.04059405940594
Config 3:
Training split 1
Loss: 2.367751889842572
Accuracy: 51.88%
Training split 

In [7]:
print(max(mean_accuracies))

51.84455445544555
